# Cards Classification


In [ ]:
# imports 
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transfroms
from torchvision.datasets import ImageFolder

In [ ]:
class PlayingCardDataset(Dataset):
    def __init__(self, data_dir, transform = None):
        self.data_dir = data_dir
        self.transform = transform
        self.data = ImageFolder(data_dir, transform = transform)
        
    def __getitem__(self, idx):
        return self.data[idx] # tuple (img, class)
        
    def __len__(self):
        # return the length of the dataset
        return len(self.data)
    @property
    def classes(self):
        return self.data.classes
        
        

In [ ]:
data_dir = '/kaggle/input/cards-image-datasetclassification/train'
dataset = PlayingCardDataset(data_dir)

In [ ]:
from tqdm.notebook import tqdm
for image, label in tqdm(dataset):
    pass
    

In [ ]:
target_to_class = {v : k for k , v in ImageFolder(data_dir).class_to_idx.items()}
print(target_to_class)

In [ ]:
transform = transfroms.Compose([
    transfroms.Resize((128, 128)),
    transfroms.ToTensor(),
])

dataset = PlayingCardDataset(data_dir, transform = transform)

In [ ]:
dataset[0][0].shape # channel,height, width

## DataLoaders

In [ ]:
dataloader = DataLoader(dataset, batch_size = 32,
                       shuffle = True)

In [ ]:
for image, labels in dataloader:
    break

In [ ]:
len(dataloader) # num of batches


In [ ]:
image.shape # batch_size, channel, height, width

In [ ]:
labels

## Model

In [ ]:
import torch.nn as nn
import torch.optim as optim
import timm

In [ ]:
base_model = timm.create_model("efficientnet_b0", pretrained = True)

In [ ]:
class SimpleCardClassifier(nn.Module):
    def __init__(self, num_classes = 53):
        super().__init__()
        self.base_model = timm.create_model("efficientnet_b0", pretrained = True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])
        # output size 1280
        feature_size = 1280
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(feature_size, num_classes)
        )
        
        
    def forward(self,x):
        # what connect the layers
        x = self.features(x)
        x = self.classifier(x)
        return x
    
model = SimpleCardClassifier()
        

In [ ]:
model(image).shape #[batch_size, num_classes]

In [ ]:
transform = transfroms.Compose([
    transfroms.Resize((128, 128)),
    transfroms.ToTensor(),
])

train_folder = '/kaggle/input/cards-image-datasetclassification/train'
valid_folder = '/kaggle/input/cards-image-datasetclassification/valid'
test_folder = '/kaggle/input/cards-image-datasetclassification/test'

train_dataset = PlayingCardDataset(train_folder, transform = transform)
val_dataset = PlayingCardDataset(valid_folder, transform = transform)
test_dataset = PlayingCardDataset(test_folder, transform = transform)

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle =True)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False)

## Traning the Model
- Pytorch Training Loop

In [ ]:
# 2 things we need to define

#loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
num_epochs = 5
train_losses = []
val_losses = []

model = SimpleCardClassifier()
model.to(device)

# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr = 0.001)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(train_loss)
    
    
    # validation phase
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
    val_loss = running_loss / len(val_loader.dataset)
    val_losses.append(val_loss)
    
    print(f"Epoch {epoch + 1} / {num_epochs} - Train loss : {train_loss} Validation loss : {val_loss}")
    
    
    
        
        
        
    

## plot the losses

In [ ]:
plt.plot(train_losses , label = 'Training loss')
plt.plot(val_losses, label = "Validation loss")
plt.legend()
plt.title("Loss over epochs")
plt.show()

## predict

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# load and preprocess the image
def preproces_image(image_path, transform):
    image = Image.open(image_path).convert("RGB")
    return image, transform(image).unsqueeze(0)

# predict using the model
def predict(model, image_tensor, device):
    model.eval()
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim = 1)
        
    return probabilities.cpu().numpy().flatten()

# visualization
def visualize_predictions(original_image, probabilities, class_names):
    fig, axarr = plt.subplots(1,2, figsize = (14,7))
    
    # display image
    axarr[0].imshow(original_image)
    axarr[0].axis('off')
    
    # display prediction 
    axarr[1].barh(class_names, probabilities)
    
    pred = np.argmax(probabilities)
    pred_class = target_to_class[pred]
    axarr[1].set_label("Probability")
    axarr[1].set_title(f"Class Prediction '{pred_class}'")
    axarr[1].set_xlim(0,1)

    plt.tight_layout()
    plt.show()
    

# example usage

#test_image = "/kaggle/input/cards-image-datasetclassification/test/ace of clubs/2.jpg"
test_image = '/kaggle/working/44636302-poker-playing-card-ace-club.jpg'
transform = transfroms.Compose([
    transfroms.Resize((128, 128)),
    transfroms.ToTensor(),
])

original_image, image_tensor = preproces_image(test_image, transform)
probabilities = predict(model, image_tensor, device)

# assuming dataset.classes gives the class names
class_names = dataset.classes
visualize_predictions(original_image, probabilities, class_names)
    
        

In [ ]:
test_filenames = glob('/kaggle/input/cards-image-datasetclassification/test/*/*.jpg')

In [ ]:
for file in np.random.choice(test_filenames , 10):
    original_image, image_tensor = preproces_image(file, transform)
    probabilities = predict(model, image_tensor, device)

    # assuming dataset.classes gives the class names
    class_names = dataset.classes
    visualize_predictions(original_image, probabilities, class_names)
